## | 🎯 Perguntas Norteadoras de Negócio
* Quais características mais influenciam na rotatividade de funcionários?
* Qual perfil de colaborador tem maior propensão a sair da empresa?
* Um modelo de machine learning pode prever com boa precisão a saída de um
funcionário?
* Que ações a empresa pode tomar com base nessas previsões?


## | 🔍 DATASET OVERVIEW

### 📂 Variáveis

* **Age:** Idade do funcionário (em anos).  
* **Attrition:** Indica se o funcionário deixou a empresa (Yes/No).  
* **BusinessTravel:** Frequência de viagens a trabalho (ex: "Travel_Rarely", "Travel_Frequently", "Non-Travel").  
* **DailyRate:** Diária, valor de um dia de trabalho(valor numérico).  
* **Department:** Departamento do funcionário (ex: "Sales", "Research & Development", "Human Resources").  
* **DistanceFromHome:** Distância entre casa e trabalho (em km).  
* **Education:** Nível educacional (1-5, onde 1="Below College", 5="Doctor").  
* **EducationField:** Área de formação (ex: "Life Sciences", "Medical", "Technical Degree").  
* **EmployeeCount:** Contagem de funcionários (normalmente 1 para registros individuais).  
* **EmployeeNumber:** ID único do funcionário.  
* **EnvironmentSatisfaction:** Satisfação com o ambiente de trabalho (escala numérica, normalmente 1-4).  
* **Gender:** Gênero ("Male" ou "Female").  
* **HourlyRate:** Remuneração por hora.  
* **JobInvolvement:** Engajamento no trabalho (escala numérica, ex: 1-4).  
* **JobLevel:** Nível hierárquico (1=júnior, 5=sênior).  
* **JobRole:** Cargo ocupado (ex: "Sales Executive", "Research Scientist").  
* **JobSatisfaction:** Satisfação com o trabalho (escala numérica, ex: 1-4).  
* **MaritalStatus:** Estado civil ("Single", "Married", "Divorced").  
* **MonthlyIncome:** Salário mensal bruto.  
* **MonthlyRate:** Salário mensal base.  
* **NumCompaniesWorked:** Número de empresas onde já trabalhou.  
* **Over18:** Se é maior de 18 anos (normalmente "Yes" para todos).  
* **OverTime:** Faz horas extras ("Yes" ou "No").  
* **PercentSalaryHike:** Percentual do último aumento salarial.  
* **PerformanceRating:** Avaliação de desempenho (ex: 1-5).  
* **RelationshipSatisfaction:** Satisfação com relacionamentos no trabalho (escala numérica).  
* **StandardHours:** Carga horária padrão (ex: 80 horas/mês).  
* **StockOptionLevel:** Nível de opções de ações (ex: 0-3).  
* **TotalWorkingYears:** Total de anos de experiência profissional.  
* **TrainingTimesLastYear:** Número de treinamentos no último ano.  
* **WorkLifeBalance:** Equilíbrio vida-trabalho (escala numérica).  
* **YearsAtCompany:** Tempo na empresa atual (em anos).  
* **YearsInCurrentRole:** Tempo no cargo atual (em anos).  
* **YearsSinceLastPromotion:** Tempo desde a última promoção (em anos).  
* **YearsWithCurrManager:** Tempo com o mesmo gerente (em anos).




### 📂 Referência

**Title**: IBM HR Analytics Employee Attrition & Performance  
**Source**: [Kaggle](https://www.kaggle.com/datasets/pavansubhasht/ibm-hr-analytics-attrition-dataset/data)  
**Author**: pavansubhash  
**License**: Database Contents License (DbCL) v1.0





## | 📚 IMPORT DE BIBLIOTECA

In [1]:
# Silenciar WARNINGS
import warnings
warnings.filterwarnings('ignore')

# Biblioteca para montar drive no colab
from google.colab import drive



# Biblioteca para manipulação dos dados
import pandas as pd
import numpy as np

# Bibliotecas para testes estatísticos
from scipy.stats import shapiro, skew
from statsmodels.stats.outliers_influence import variance_inflation_factor



# Bibliotecas para visualização
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import panel as pn
import plotly.express as px

# Biblioteca para aprendizado de máquina
import sklearn
from sklearn.preprocessing import OneHotEncoder


## | 📂 LEITURA DE DADOS

In [2]:
# Cria drive no colab
drive.mount('/content/drive', force_remount=True)

# Especifica caminho do arquivo a ser lido
caminho = '/content/drive/MyDrive/bancos/RH-DATASET.csv'

Mounted at /content/drive


In [3]:
# Instancia variável atribuindo arquivo
df = pd.read_csv(caminho)

## | ☑️ DATA WRANGLING

In [4]:
# Visualizar 5 primeiras linhas
df.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


In [5]:
# Visualiza as colunas do datset
df.columns

Index(['Age', 'Attrition', 'BusinessTravel', 'DailyRate', 'Department',
       'DistanceFromHome', 'Education', 'EducationField', 'EmployeeCount',
       'EmployeeNumber', 'EnvironmentSatisfaction', 'Gender', 'HourlyRate',
       'JobInvolvement', 'JobLevel', 'JobRole', 'JobSatisfaction',
       'MaritalStatus', 'MonthlyIncome', 'MonthlyRate', 'NumCompaniesWorked',
       'Over18', 'OverTime', 'PercentSalaryHike', 'PerformanceRating',
       'RelationshipSatisfaction', 'StandardHours', 'StockOptionLevel',
       'TotalWorkingYears', 'TrainingTimesLastYear', 'WorkLifeBalance',
       'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion',
       'YearsWithCurrManager'],
      dtype='object')

In [6]:
# Visualiza informações
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 35 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Age                       1470 non-null   int64 
 1   Attrition                 1470 non-null   object
 2   BusinessTravel            1470 non-null   object
 3   DailyRate                 1470 non-null   int64 
 4   Department                1470 non-null   object
 5   DistanceFromHome          1470 non-null   int64 
 6   Education                 1470 non-null   int64 
 7   EducationField            1470 non-null   object
 8   EmployeeCount             1470 non-null   int64 
 9   EmployeeNumber            1470 non-null   int64 
 10  EnvironmentSatisfaction   1470 non-null   int64 
 11  Gender                    1470 non-null   object
 12  HourlyRate                1470 non-null   int64 
 13  JobInvolvement            1470 non-null   int64 
 14  JobLevel                

In [7]:
# Checa se há ids repetidos
df['EmployeeNumber'].duplicated().value_counts()

,count
EmployeeNumber,
False,1470


In [8]:
# Dicionário para entender a cardinalidade, verificar se os tipos estão corretos e entender os valores de baixa cardinalidade

# Imprime valores únicos caso cardinalidade seja no máximo 15

dic = pd.DataFrame({
    'Tipo': df.dtypes,
    'Total Valores únicos': df.nunique(),
    'Valores mais frequentes': df.mode().iloc[0],
    'Valores únicos': df.apply(lambda x: list(x.unique()) if x.nunique()<=15 else "N/A")
})

print('🗃️ Resumo de informações:')
dic

🗃️ Resumo de informações:


,Tipo,Total Valores únicos,Valores mais frequentes,Valores únicos
Age,int64,43,35.0,N/A
Attrition,object,2,No,"[Yes, No]"
BusinessTravel,object,3,Travel_Rarely,"[Travel_Rarely, Travel_Frequently, Non-Travel]"
DailyRate,int64,886,691.0,N/A
Department,object,3,Research & Development,"[Sales, Research & Development, Human Resources]"
DistanceFromHome,int64,29,2.0,N/A
Education,int64,5,3.0,"[2, 1, 4, 3, 5]"
EducationField,object,6,Life Sciences,"[Life Sciences, Other, Medical, Marketing, Tec..."
EmployeeCount,int64,1,1.0,[1]
EmployeeNumber,int64,1470,1,N/A


In [9]:
# Função para testar a normalidade das colunas numéricas
def test_normalidade(coluna):
  w_stat,p_value = shapiro(coluna)
  return "Normal" if p_value > 0.05 else "Não normal"

# Seleciona colunas numéricas
num = df.select_dtypes(include=['number'])

# Testa normalidade em cada coluna com apply()
dis_normal = num.apply(test_normalidade)

# Calcula assimetria (skewness) em cada coluna
assimetria = num.skew().round(4)

# Dataframe com as informaçções por coluna
resumo = pd.DataFrame({
    'Distribuição (Shapiro-Wilk)': dis_normal,
    ' Assimetria': assimetria
})

resumo

,Distribuição (Shapiro-Wilk),Assimetria
Age,Não normal,0.4133
DailyRate,Não normal,-0.0035
DistanceFromHome,Não normal,0.9581
Education,Não normal,-0.2897
EmployeeCount,Normal,0.0000
EmployeeNumber,Não normal,0.0166
EnvironmentSatisfaction,Não normal,-0.3217
HourlyRate,Não normal,-0.0323
JobInvolvement,Não normal,-0.4984
JobLevel,Não normal,1.0254


### 📑 Resumo executivo Data Wrangling

* 35 colunas e 1470 linhas
* Não há 'missing values'
* Não há ids ('EmployeeNumber') repetidos
* Colunas numéricas: apenas duas tem distribuição normal, as demais possuem distribuição assimétrica, majoritariamente positivo


#### 🟩 Classificação das variáveis

* **Variáveis categóricas nominal:** 'Attrition' (Variável target), 'Department', 'EducationField', 'EmployeeNumber', 'Gender', 'JobRole', 'MaritalStatus', 'Over18', 'OverTime',
* **Variáveis categóricas ordinal:** 'BusinessTravel', 'Education', 'EnvironmentSatisfaction', 'JobInvolvement', 'JobSatisfaction','JobLevel', 'PerformanceRating', 'RelationshipSatisfaction', 'WorkLifeBalance', 'StockOptionLevel'.
* **Variáveis numéricas contínuas:** 'DistanceFromHome', 'MonthlyIncome', 'MonthlyRate',
* **Variáveis numéricas discretas:** 'Age', 'DailyRate', 'EmployeeCount', 'HourlyRate', 'NumCompaniesWorked', 'PercentSalaryHike', 'StandardHours', 'TotalWorkingYears', 'TrainingTimesLastYear', 'YearsAtCompany' 'YearsInCurrentRole', 'YearsSinceLastPromotion', 'YearsWithCurrManager'.

#### 🟩 Tratamentos aplicados

* 'StockOptionLevel' o nível disponível para escolha é definido pela empresa, que usualmente defini os requisitos baseados em senioridade e até performance, por isto classifiquei como categórica ordinal
* As colunas 'EmployeeCount', 'Over18', 'StandardHours' apresentam valores constantes, sendo desinteressantes para a análise, por isto serão removidas
*  'EmployeeNumber' é o id de identificação, também será removido
* Transformação dos dtypes de todas as variáves categóricas para 'category"

In [10]:
# Faz cópia do df original
df_tratado= df.copy()

# Dropa colunas não informativas
df_tratado = df_tratado.drop(columns=['EmployeeCount', 'Over18', 'StandardHours','EmployeeNumber'], axis=1)

In [11]:
# Transforma dtype das variáveis categóricas nominais

# Lista de colunas nominais
colunas_nominais = [
    'Attrition', 'Department', 'EducationField', 'Gender',
    'JobRole', 'MaritalStatus', 'OverTime', 'BusinessTravel'
]

df_tratado[colunas_nominais] = df_tratado[colunas_nominais].astype('category')

In [12]:
# Transforma dtype das variáveis categóricas ordinais

# Lista de colunas ordinais
# Todas as colunas ordinais são numéricas
colunas_ordinais = [
    'Education', 'EnvironmentSatisfaction', 'JobInvolvement', 'JobSatisfaction','JobLevel', 'PerformanceRating', 'RelationshipSatisfaction', 'WorkLifeBalance', 'StockOptionLevel'
]

# Transforma Dtypes preservando a hierarquia dentro de cada feature ordinal
for column in colunas_ordinais:
  df_tratado[column] = pd.Categorical(df_tratado[column], categories = sorted(df[column].unique()), ordered= True)


In [13]:
# Checa transformações nos tipos de variáveis
df_tratado.dtypes

,0
Age,int64
Attrition,category
BusinessTravel,category
DailyRate,int64
Department,category
DistanceFromHome,int64
Education,category
EducationField,category
EnvironmentSatisfaction,category
Gender,category


## | 📊 ANÁLISE EXPLORATÓRIA


#### **Análise exploratória univariada**

In [14]:
# Imprime dados estatísticos das variáveis numéricas
df_tratado.describe(include=['number'])

,Age,DailyRate,DistanceFromHome,HourlyRate,MonthlyIncome,MonthlyRate,NumCompaniesWorked,PercentSalaryHike,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
count,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000
mean,36.923810,802.485714,9.192517,65.891156,6502.931293,14313.103401,2.693197,15.209524,11.279592,2.799320,7.008163,4.229252,2.187755,4.123129
std,9.135373,403.509100,8.106864,20.329428,4707.956783,7117.786044,2.498009,3.659938,7.780782,1.289271,6.126525,3.623137,3.222430,3.568136
min,18.000000,102.000000,1.000000,30.000000,1009.000000,2094.000000,0.000000,11.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,30.000000,465.000000,2.000000,48.000000,2911.000000,8047.000000,1.000000,12.000000,6.000000,2.000000,3.000000,2.000000,0.000000,2.000000
50%,36.000000,802.000000,7.000000,66.000000,4919.000000,14235.500000,2.000000,14.000000,10.000000,3.000000,5.000000,3.000000,1.000000,3.000000
75%,43.000000,1157.000000,14.000000,83.750000,8379.000000,20461.500000,4.000000,18.000000,15.000000,3.000000,9.000000,7.000000,3.000000,7.000000
max,60.000000,1499.000000,29.000000,100.000000,19999.000000,26999.000000,9.000000,25.000000,40.000000,6.000000,40.000000,18.000000,15.000000,17.000000


In [15]:
# Imprime dados das frequências das variáveis categóricas
df_tratado.describe(include=['category'])

,Attrition,BusinessTravel,Department,Education,EducationField,EnvironmentSatisfaction,Gender,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,OverTime,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,WorkLifeBalance
count,1470,1470,1470,1470,1470,1470,1470,1470,1470,1470,1470,1470,1470,1470,1470,1470,1470
unique,2,3,3,5,6,4,2,4,5,9,4,3,2,2,4,4,4
top,No,Travel_Rarely,Research & Development,3,Life Sciences,3,Male,3,1,Sales Executive,4,Married,No,3,3,0,3
freq,1233,1043,961,572,606,453,882,868,543,326,459,673,1054,1244,459,631,893


In [16]:
# Inicializa paleta de cores para reutilização nos gráficos
palette = {
    # Cores da interface (mantive o fundo claro para legibilidade)
    'background': '#FFFFFF',
    'text': '#08415C', # Usando o azul escuro para o texto principal
    'grid': '#D3D3D3', # Um cinza claro para as grades

    # Cores principais para dados e destaques
    'primary': '#08415C',    # Indigo dye
    'accent': '#CC2936',     # Persian red
    'secondary': '#F1BF98',  # Peach
    'gray1': '#6B818C',      # Slate gray

    # Cores para linhas estatísticas (média, mediana)
    'mean': '#CC2936',       # O vermelho para dar bastante destaque à média
    'median': '#6B818C'      # O cinza ardósia para a mediana
}

In [17]:
# Garante que o Plotly funcione bem com o Panel aqui no Colab
pn.extension('plotly')

# Definindo as listas de colunas
colunas_numericas = df_tratado.select_dtypes(include=np.number).columns.tolist()
colunas_categoricas = df_tratado.select_dtypes(include=['category']).drop('Gender', axis=1, errors='ignore').columns.tolist()


# Cria Widget para filtrar por gênero
gender_filter = pn.widgets.Select(
    name='Filtrar por gênero',
    options=['All'] + list(df['Gender'].unique())
)

# Cria widget para selecionar a variável a ser analisada
variable_selector = pn.widgets.Select(
    name='Selecionar Variável',
    options=colunas_numericas + colunas_categoricas
)


# O decorator agora "escuta" as mudanças nos dois widgets
@pn.depends(gender_filter.param.value, variable_selector.param.value)
def update_plots(gender_val, variable_val):

    # Filtra o DataFrame com base no gênero
    filtered_df = df.copy()
    if gender_val != 'All':
        filtered_df = filtered_df[filtered_df['Gender'] == gender_val]

    # LÓGICA IF/ELSE PARA TRATAR VARIÁVEIS NUMÉRICAS E CATEGÓRICAS
    # SE A VARIÁVEL FOR NUMÉRICA
    if variable_val in colunas_numericas:
        # Cálculo estatístico para a variável selecionada
        stats = {
            'mean': filtered_df[variable_val].mean(),
            'median': filtered_df[variable_val].median()
        }

        # 1. Histograma (generalizado para a variável selecionada)
        hist = px.histogram(
            filtered_df, x=variable_val, nbins=30,
            color=None if gender_val != 'All' else 'Gender',
            color_discrete_map={'Female': palette['primary'], 'Male': palette['accent']},
            title=f'<b style="color:{palette["primary"]}">Distribuição de {variable_val}</b>'
        )

        # Adiciona a linha da MÉDIA
        hist.add_vline(x=stats['mean'], line=dict(color=palette['mean'], width=2, dash='dash'),
                        annotation_text=f"Média: {stats['mean']:.2f}")

        # 2. Boxplot (generalizado para a variável selecionada)
        box = px.box(
            filtered_df, y=variable_val,
            x=None if gender_val != 'All' else 'Gender',
            color=None if gender_val != 'All' else 'Gender',
            color_discrete_map={'Female': palette['primary'], 'Male': palette['accent']},
            title=f'<b style="color:{palette["primary"]}">Boxplot de {variable_val}</b>'
        )

        return pn.Row(hist, box, sizing_mode='stretch_width')

    # SE A VARIÁVEL FOR CATEGÓRICA
    elif variable_val in colunas_categoricas:
        # Calcula a contagem de cada categoria
        counts = filtered_df[variable_val].value_counts().reset_index()
        counts.columns = ['Categoria', 'Contagem']

        # Cria o gráfico de barras
        bar_chart = px.bar(
            counts, x='Categoria', y='Contagem', text_auto=True,
            title=f'<b style="color:{palette["primary"]}">Distribuição de {variable_val}</b>'
        )
        return bar_chart


# Monta dashboard final
dashboard = pn.Column(
    pn.Row(
        pn.pane.Markdown("## Análise de Desempenho Acadêmico"),
        gender_filter,
        variable_selector, # Novo dropdown adicionado aqui
        styles={'background': palette['background'], 'padding': '10px'}
    ),
    update_plots,
    styles={'background': 'white', 'border': f"1px solid {palette['gray1']}"},
    sizing_mode='stretch_width'
)

dashboard.servable()

Column(sizing_mode='stretch_width', styles={'background': 'white', ...})
    [0] Row(styles={'background': '#FFFFFF', ...})
        [0] Markdown(str)
        [1] Select(name='Filtrar por gênero', options=['All', 'Female', 'Male'], value='All')
        [2] Select(name='Selecionar Variável', options=['Age', 'DailyRate', ...], value='Age')
    [1] ParamFunction(function, _pane=Row, defer_load=False)

In [18]:
# Calcula % de gênero
df_tratado['Gender'].value_counts(normalize=True)

,proportion
Gender,
Male,0.6
Female,0.4


In [19]:
# Visualizando os detalhes de pessoas que responderam 0 para o número de companias trabalhadas
df_tratado[df_tratado['NumCompaniesWorked']==0]

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,Gender,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
5,32,No,Travel_Frequently,1005,Research & Development,2,2,Life Sciences,4,Male,...,3,3,0,8,2,2,7,7,3,6
8,38,No,Travel_Frequently,216,Research & Development,23,3,Life Sciences,4,Male,...,4,2,0,10,2,3,9,7,1,8
10,35,No,Travel_Rarely,809,Research & Development,16,3,Medical,1,Male,...,3,3,1,6,5,3,5,4,0,3
11,29,No,Travel_Rarely,153,Research & Development,15,2,Life Sciences,4,Female,...,3,4,0,10,3,3,9,5,0,8
13,34,No,Travel_Rarely,1346,Research & Development,19,2,Medical,2,Male,...,3,3,1,3,2,3,2,2,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1439,36,No,Travel_Rarely,557,Sales,3,3,Medical,1,Female,...,3,3,2,10,2,3,9,7,3,4
1445,41,No,Travel_Rarely,582,Research & Development,28,4,Life Sciences,1,Female,...,4,3,1,21,3,3,20,7,0,10
1462,39,No,Travel_Rarely,722,Sales,24,1,Marketing,2,Female,...,3,1,1,21,2,2,20,9,9,6
1463,31,No,Non-Travel,325,Research & Development,5,3,Medical,2,Male,...,3,2,0,10,2,3,9,4,1,7


In [20]:
# Teste Mann-Whitney U para comparar medianas dos salários




##### 📑 Resumo Executivo - Análise Exploratória Univariada

* 60% gênero masculino e 40% gênero feminino.

* Idade: varia de 18 a 60 anos, com mediana de 36 anos.

* Diárias (DailyRate): a mediana para o gênero feminino (825.5) é maior que a mediana para o gênero masculino (783), (VERIFICAR SE HÁ DIFERENÇA ESTATÍSTICA ENTRE OS VALORES).

* Distância de casa: os dois grupos tem distribuição similar, 75% moram a no mínimo 14 unidades de distância do trabalho, e 50% a menos de 7.

* Salário por hora (HourlyRate) e salário mensal (MonthlyIncome): a mediana do grupo feminino é levemente superior que a mediana do grupo masculino para as duas métricas, 67 contra 65 e 5.081,5 contra 4.837,5. Como esperado para distribuição de salário, os salários altos puxam a assimetria para a direita.

* Número de companias trabalhadas: assumi que a coluna se refere a companias diferentes da atual, sendo assim, por adoção aquelas colunas atribuídas com 0 referem-se as pessoas que trabaharam apenas na compania em que foi realizada a pesquisa. No conjunto de dados avaliados, as mulheres tem mediana(2) de companias maior que a dos homens(1).

* A mediana de aumento salarial é igual para os dois gêneros (~14%).

* A análise de tempo na função atual mostra que o terceiro quartil (Q3) da distribuição é de 7 anos, indicando que 75% dos colaboradores estão em seus cargos por um período igual ou inferior a este. Ao segmentar por gênero, observa-se que não há diferença na tendência central, pois a mediana de tempo no cargo é a mesma para homens e mulheres. No entanto, ao analisar os anos desde a última promoção, a diferença entre os gêneros se torna evidente. Para 75% dos homens, a última promoção ocorreu em um período de até 2 anos, enquanto para 75% das mulheres, esse mesmo período se estende para até 3 anos.

* Analisando o perfil de evasão, observa-se uma disparidade de gênero: a proporção de homens que deixaram a empresa (63%) é consideravelmente maior que a de mulheres (37%).

* O departamento de 'Research & Development' concentra a maioria dos funcionários (65%), seguido por 'Sales' (30%) e 'Human Resources' com os 5% restantes.

* Um ponto de atenção é a baixa satisfação geral, tanto com o relacionamento interpessoal quanto com o trabalho em si, ambas reportadas por 39% dos colaboradores. Ao analisar o perfil desses grupos insatisfeitos, nota-se uma predominância masculina. Na baixa satisfação com o relacionamento, os homens representam 60% dos impactados, enquanto na insatisfação com o trabalho, eles compõem 59% do grupo.




####  **Análise exploratória multivariada**


* Pré-processamento

In [21]:
#  One-Hot Encoding nas colunas nominais

# Instancia objeto com drop da primeira categoria pra economizar memória
ohe = OneHotEncoder(drop='first', sparse_output=False, dtype=np.int64)

# Aplica transformação
dados_ohe = ohe.fit_transform(df_tratado[colunas_nominais])

# Cria dataframe com os nomes corretos e mantem o índice pra evitar problemas
df_dummies = pd.DataFrame(dados_ohe, columns = ohe.get_feature_names_out(colunas_nominais), index=df_tratado.index)

# Concatena variáveis dummy com df removendo nominais
df_final = pd.concat([df_tratado.drop(columns=colunas_nominais), df_dummies], axis=1)


* Visualização de multivariada

In [ ]:
sns.displot(data=df_final)

In [ ]:
# Checa operação anterior
df_final.head()

In [ ]:
df_final.columns

In [ ]:
# Matriz de correlação
corr_matriz = df_final.corr(method= 'spearman')

print( " Matriz de correlação")
corr_matriz


In [ ]:
# Investiga maiores corlinearidades bivariadas

# Obtem valores absolutos na matriz de covariância
corr_matriz_abs = corr_matriz.abs()

# Desempilha a matriz para criar uma lista de pares de correlação
corr_unstacked = corr_matriz_abs.unstack()

# Ordene a lista para ver os maiores valores no topo
corr_sorted = corr_unstacked.sort_values(ascending=False)

# 4. Filtre para remover a correlação de uma variável com ela mesma (que é sempre 1), remova os duplicados (a correlação de A com B é a mesma de B com A)
top_corr_pairs = corr_sorted[corr_sorted < 1].drop_duplicates()


print("--- Top 30 Pares Mais Correlacionados no Dataset ---")
print(top_corr_pairs.head(40))

In [ ]:
# Determinar Fator de Inflação de Variância (VIF), multicolinearidade, entre as variáveis independentes
# Quanto a variância de um coeficiente é inflada pelas relações com outras variáveis
vif_data = pd.DataFrame()
vif_data["feature"] = df_final.columns
vif_data["VIF"] = [variance_inflation_factor(df_final.values, i) for i in range(len(df_final.columns))]

In [ ]:
# Ordena df com VIF
vif_data_sorted = vif_data.sort_values(by='VIF', ascending=False)

print(" Multicolinearidade como VIF")
print(vif_data_sorted)

- Várias colinearidades com valores significantes: tratar!
- Multicolinearidade reduz o poder estatístico de um modelo
- Em datasets com multicolineariade, pequenas alterações nos dados podem levar a grandes alterações nas estimativas dos parâmetros, até mesmo inversões de sinal.

Fontes consultadas:
https://stats.stackexchange.com/tags/multicollinearity/info

https://statisticsbyjim.com/regression/multicollinearity-in-regression-analysis/



In [ ]:
# Correlação entre target e demais features
# Calcula correlaçã de spearman
corr_table = df_final.corr(method='spearman')['Attrition_Yes']

# Ordena tabela de correlação
corr_ordenada = corr_table.sort_values(ascending=False)

print("--- Correlação de cada variável com 'Attrition_Yes' ---")
print(corr_ordenada.drop('Attrition_Yes'))

# Cria um gráfico de barras para visualizar melhor
plt.figure(figsize=(8, 10))

# Remove a correlação da própria variável alvo (sempre 1) para o gráfico
corr_ordenada.drop('Attrition_Yes').plot(kind='barh')
plt.title("Correlação de Spearman com a Variável Alvo (Attrition_Yes)")
plt.grid(axis='x', linestyle='--')
plt.show()

In [ ]:
# Determinar Fator de Inflação de Variância (VIF): multicolinearidade
# Quanto a variância de um coeficiente é inflada pelas relações com outras variáveis

df_sem_multicolieres_altas = df_final.drop(columns=['Age', 'PerformanceRating', 'TotalWorkingYears', 'JobInvolvement','MonthlyIncome', 'EducationField_Medical', 'EducationField_Other', 'YearsWithCurrManager', 'DailyRate', 'EnvironmentSatisfaction', 'BusinessTravel_Travel_Rarely', 'Education', 'RelationshipSatisfaction', 'JobRole_Sales Executive', 'Department_Research & Development', 'YearsAtCompany', 'HourlyRate', 'WorkLifeBalance', 'JobLevel', 'PercentSalaryHike', 'TrainingTimesLastYear', 'JobSatisfaction'   ], axis=1)

vif_data = pd.DataFrame()
vif_data["feature"] = df_sem_multicolieres_altas.columns
vif_data["VIF"] = [variance_inflation_factor(df_sem_multicolieres_altas.values, i) for i in range(len(df_sem_multicolieres_altas.columns))]

# Ordena df com VIF
vif_data_sorted = vif_data.sort_values(by='VIF', ascending=False)

print(" Multicolinearidade como VIF")
print(vif_data_sorted)

In [ ]:
vif_data_sorted.feature.values

In [ ]:
variaveis_vif_baixo = df_final[['MonthlyRate', 'Department_Sales', 'YearsInCurrentRole', 'MaritalStatus_Single', 'StockOptionLevel',
       'MaritalStatus_Married', 'JobRole_Research Scientist',
       'Gender_Male', 'DistanceFromHome', 'JobRole_Laboratory Technician',
       'YearsSinceLastPromotion', 'NumCompaniesWorked',
       'EducationField_Life Sciences', 'EducationField_Marketing',
       'JobRole_Manufacturing Director', 'OverTime_Yes', 'Attrition_Yes',
       'JobRole_Research Director', 'JobRole_Sales Representative','JobRole_Manager', 'BusinessTravel_Travel_Frequently', 'EducationField_Technical Degree', 'JobRole_Human Resources']]

In [ ]:
# Correlação entre target e demais features
# Calcula correlaçã de spearman
corr_table = variaveis_vif_baixo.corr(method='spearman')['Attrition_Yes']

# Ordena tabela de correlação
corr_ordenada = corr_table.sort_values(ascending=False)

print("--- Correlação de cada variável com 'Attrition_Yes' ---")
print(corr_ordenada.drop('Attrition_Yes'))

# Cria um gráfico de barras para visualizar melhor
plt.figure(figsize=(8, 10))

# Remove a correlação da própria variável alvo (sempre 1) para o gráfico
corr_ordenada.drop('Attrition_Yes').plot(kind='barh')
plt.title("Correlação de Spearman com a Variável Alvo (Attrition_Yes)")
plt.grid(axis='x', linestyle='--')
plt.show()

In [ ]:
# Visualiza quais cargos, departamentos estão associados a maiores hikes

In [ ]:
# Investigar jobsatsfaction X 'EnvironmentSatisfaction' X 'RelationshipSatisfaction'

#'HourlyRate': TALVEZ CALCULAR O % DE HORAS TRABAHADAS DO TOTAL DE 80h

Dividir grupos entre baixa satisfação (1-2)
Média satisfação (3 )
Satifação alta (4)
 e entender o número de atrition por grupo

Composição de atrition por gênero

Os maiores salários são pra quais áreas, generos ?
 Distribuição e assimetria

## | ✅ PRÉ-PROCESSAMENTO DOS DADOS PARA MODELAGEM

### Subtask:
Prepare the data for machine learning models. This will involve encoding categorical variables and splitting the data into training and testing sets.


## | 💻 MODELAGEM PREDITIVA

## | 📋 AVALIAÇÃO DOS MODELOS

## | 📍 CONCLUSÃO